In [132]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import *

In [133]:
spark = SparkSession.builder.master("local[*]").appName("agg").getOrCreate()

In [135]:
map(lambda x: x + 1, [1, 2, 3])

In [134]:
df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("../data/retail-data/all/*.csv")
    .coalesce(5)
)
df.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: int, Country: string]

In [ ]:
df.count()

In [ ]:
df.select(count("StockCode").alias("count_stock_code")).show()

In [ ]:
df.select(count_distinct("StockCode").alias("count_stock_code")).show()

In [ ]:
df.select(approx_count_distinct("StockCode", 0.03).alias("count_stock_code")).show()

In [ ]:
df.select(first("StockCode"), last("StockCode")).show()

In [ ]:
df.select(min("Quantity"), max("Quantity")).show()

In [ ]:
df.select(sum("Quantity")).show()

In [ ]:
df.select(sum_distinct("Quantity")).show()

In [ ]:
df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases"),
).selectExpr(
    "(total_purchases/total_transactions) as ratio_purchases_transactions",
    "avg_purchases",
    "mean_purchases",
).show()

In [ ]:
df.show(5)

In [ ]:
df.agg(collect_set("Country"), collect_list("Country")).show()

grouping

In [ ]:
(
    df.where(col("CustomerId").isNotNull())
    .groupBy("InvoiceNo", "CustomerId")
    .count()
    .orderBy(desc("count"))
).show()

In [ ]:
(df.groupBy("InvoiceNo").agg(count_distinct("Quantity").alias("quan"))).show()

In [ ]:
pessoas = (
    df.groupBy("InvoiceNo")
    .agg(sum("Quantity").alias("sum_quantity"))
    .where(col("sum_quantity") > 0)
    .dropDuplicates()
)

In [ ]:
pessoas.agg(max(col("sum_quantity")), min(col("sum_quantity"))).show()

In [ ]:
w = Window.orderBy(desc("sum_quantity"))

pessoas_10_pct = pessoas.withColumn("10_porcento", ntile(10).over(w))

In [ ]:
# pessoas_10_pct.orderBy(desc("sum_quantity")).show()


pessoas_10_pct.where(col("10_porcento") == 1).agg(min(col("sum_quantity"))).show()

In [ ]:
df_with_date = df.withColumn("date", to_date(col("InvoiceDate"), "M/d/y H:mm"))

In [ ]:
df_with_date.show()

In [ ]:
windowSpec = (
    Window.partitionBy("CustomerID", "date")
    .orderBy(desc("Quantity"))
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)
purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)
sumPurchase = sum(("Quantity")).over(windowSpec)

In [ ]:
df_with_date.where("CustomerId IS NOT NULL").orderBy("CustomerId").select(
    col("CustomerId"),
    col("date"),
    col("Quantity"),
    purchaseRank.alias("quantity_rank"),
    purchaseDenseRank.alias("quantity_dense_rank"),
    maxPurchaseQuantity.alias("max_purchase_quantity"),
    sumPurchase.alias("sum_quantity"),
).show()

In [ ]:
pivoted = df_with_date.groupBy("date").pivot("Country").sum()

In [ ]:
pivoted.filter('date > "2011-12-05"').select("date", "`USA_sum(Quantity)`").show()

join

In [ ]:
person = spark.createDataFrame(
    [
        (0, "Bill Chambers", 0, [100]),
        (1, "Matei Zaharia", 1, [500, 250, 100]),
        (2, "Michael Armbrust", 1, [250, 100]),
    ]
).toDF("id", "name", "graduate_program", "spark_status")

graduateProgram = spark.createDataFrame(
    [
        (0, "Masters", "School of Information", "UC Berkeley"),
        (2, "Masters", "EECS", "UC Berkeley"),
        (1, "Ph.D.", "EECS", "UC Berkeley"),
    ]
).toDF("id", "degree", "department", "school")

sparkStatus = spark.createDataFrame(
    [(500, "Vice President"), (250, "PMC Member"), (100, "Contributor")]
).toDF("id", "status")

In [ ]:
from pyspark.sql.types import StructField, IntegerType

In [ ]:
schema = StructType(
    [StructField("DESC", StringType(), False), StructField("ID", IntegerType(), False)]
)

df = spark.createDataFrame([temp1], schema)

df.show()

In [ ]:
data = [
    {"Category": "A", "ID": 1, "Value": 121.44, "Truth": True},
    {"Category": "B", "ID": 2, "Value": 300.01, "Truth": False},
    {"Category": "C", "ID": 3, "Value": 10.99, "Truth": None},
    {"Category": "E", "ID": 4, "Value": 33.87, "Truth": False},
]

cdf = spark.createDataFrame(data)
type(cdf)